So this is the lab...
Logistic regression and stuff
so yeah... we need paramters for regularization
none, L1, L2, this can jsut be a flag with hyperparamters and overloaded functions for the regulairzation...
well maybe not overloading but a method that we call that calls other methods called L1 and L2
we can compare different regularizations within the same method, that way its legit. 
for the methods... Were suggetsted to the Stochiastic graident descent. mini-batch gradient descent for our steepest descent and the hessian for the quasi newton thingy. Frick do we gotta write our own hessian? Thatll suck big time

Step1.
Read data and encode locations as one hot encoded
Step2.
Split into 80% train and 20 %test after merging.
Step3. Write regularization
Do these tonight


We did both of the regularzations but not together \for reg we do ridge lasso or elastic net 
gradient[1:] = regularization + get gradient

In [126]:
#this is the block that simply reads the data
import os
import re
import math
import pandas as pd
def translate_height(height):
    height = str(height)
    if height == "nan":
        return None
    l = re.findall(r'[0-9]+',height)
    if len(l) == 1:
        return int(l[0])
    else:
        return int(l[0]) + int(l[1])*12
directory = os.path.relpath("clothing-fit-dataset-for-size-recommendation/")
print(directory)
df = []
y = []
df = pd.read_json('clothing-fit-dataset-for-size-recommendation/modcloth_final_data.json', lines=True)
di = {"very short": 1, "slightly short": 2, "just right": 3, "slightly long": 4, "very long": 5}
df = df.replace({"length": di})
di = {"small": 1, "fit": 2, "large": 3}
df = df.replace({"fit": di})
df["height"] = df["height"].map(lambda x: translate_height(x))

di = {"aa": 0, "a": 1, "b": 2, "c": 3, "d": 4, "dd/e": 5, "ddd/f": 6, "dddd/g": 7, "h": 8, "i": 9, "j": 10, "k": 11}
df = df.replace({"cup size": di})
#maybe add shoe width back?
y = df['fit']
df = df.drop(columns=["item_id", "category", "user_name", "user_id", "shoe width", "review_summary", "review_text", "fit"])
print("NaNs per column, if more than 20% we will drop otherwise")
for column in df:
    print(df[column].isna().sum(), column)
    df[column] = pd.to_numeric(df[column], errors="coerce")
df = df.drop(columns=["waist", "bust"])
df = df.fillna(df.mean())
print(df.head())
#big issue here is predicting columns with shoes? wtf there are so many NaNs. Not sure what to do about shoe products?
x = df.values.astype(int)
x


clothing-fit-dataset-for-size-recommendation
NaNs per column, if more than 20% we will drop otherwise
79908 waist
0 size
68 quality
6255 cup size
26726 hips
6018 bra size
70936 bust
1107 height
35 length
54875 shoe size
   size  quality  cup size       hips   bra size     height  length  shoe size
0     7      5.0       4.0  38.000000  34.000000  77.000000     3.0   8.145818
1    13      3.0       2.0  30.000000  36.000000  29.000000     3.0   8.145818
2     7      2.0       2.0  40.358501  32.000000  89.000000     4.0   9.000000
3    21      5.0       5.0  40.358501  35.972125  67.016136     3.0   8.145818
4    18      5.0       2.0  40.358501  36.000000  29.000000     4.0   8.145818


array([[ 7,  5,  4, ..., 77,  3,  8],
       [13,  3,  2, ..., 29,  3,  8],
       [ 7,  2,  2, ..., 89,  4,  9],
       ...,
       [12,  5,  7, ..., 53,  3,  8],
       [12,  4,  3, ..., 41,  3,  8],
       [ 4,  4,  4, ...,  6,  3,  8]])

In [127]:
#this is for the renttherunway stuff
y

0        1
1        1
2        1
3        2
4        1
        ..
82785    2
82786    1
82787    2
82788    2
82789    2
Name: fit, Length: 82790, dtype: int64

In [128]:
#now split into training and testing data
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = .80)
y_train

46339    2
46904    2
82525    2
8667     2
65509    2
        ..
6174     3
16004    2
31445    1
73549    1
25738    2
Name: fit, Length: 16558, dtype: int64

In [129]:
#test logistic regresion to see if my splits are working
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
clf = LogisticRegression(fit_intercept=False)
clf.fit(X_train, y_train)

yhat = clf.predict(X_test)
print('Accuracy of: ',accuracy_score(y_test,yhat))

/Users/haydendonofrio/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/haydendonofrio/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy of:  0.6896515279623143


## Regurlzarization


In [130]:
class BinaryLogisticRegressionBase:
    # private:
    def __init__(self, eta, iterations=20):
        self.eta = eta
        self.iters = iterations
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        return 'Base Binary Logistic Regression Object, Not Trainable'
    
    # convenience, private and static:
    @staticmethod
    def _sigmoid(theta):
        return 1/(1+np.exp(-theta)) 
    
    @staticmethod
    def _add_bias(X):
        return np.hstack((np.ones((X.shape[0],1)),X)) # add bias term
    
    # public:
    def predict_proba(self,X,add_bias=True):
        # this typically gives us the probability of a specific class
        # we need a bias term 
        # add bias term if requested
        Xb = self._add_bias(X) if add_bias else X
        # this will take the x with the bias term and multiplies it by w. This will give us a scalr quantity
        return self._sigmoid(Xb @ self.w_) # return the probability y=1
    
    def predict(self,X):
        # if we actually want to do a prediction we are seeing if it is greater than point 5. vector of ones were it is greater than .5 and 0 when it is below .5
        # When one is greater than the other it would be greater than .5
        return (self.predict_proba(X)>0.5) #return the actual prediction

In [131]:
class BinaryLogisticRegression(BinaryLogisticRegressionBase):
    #private:
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'Binary Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained Binary Logistic Regression Object'
        
    def _get_gradient(self,X,y):
        # programming \sum_i (yi-g(xi))xi
        #for each vector x we want to out it throught he sigmoid suntract it from y and add it to the gradient function
        gradient = np.zeros(self.w_.shape) # set gradient to zero
        for (xi,yi) in zip(X,y):
            # the actual update inside of sum
            gradi = (yi - self.predict_proba(xi,add_bias=False))*xi 
            # reshape to be column vector and add to gradient
            gradient += gradi.reshape(self.w_.shape) 
        #return gradient divided by the total number of points
        return gradient/float(len(y))
       
    # public:
    def fit(self, X, y):
        #this is going to get features and labels. We are going to say x wtih a bias is ther (col of 1s)
        #the shape is the # of samples (rows ) by # of features (cols)
        #w always starts at 0 usually. 
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros
        
        # for as many as the max iterations
        # whats the gradient?? and internall we are updating self.w_ in this
        
        for _ in range(self.iters):
            gradient = self._get_gradient(Xb,y)
            self.w_ += gradient*self.eta # multiply by learning rate 

           

In [132]:
from scipy.special import expit
class VectorBinaryLogisticRegression(BinaryLogisticRegression):
    # inherit from our previous class to get same functionality
    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta) #1/(1+np.exp(-theta))
    
    # but overwrite the gradient calculation
    def _get_gradient(self,X,y):
        ydiff = y-self.predict_proba(X,add_bias=False).ravel() # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        
        return gradient.reshape(self.w_.shape)

In [133]:
class LogisticRegression:
    def __init__(self, eta, iterations=20):
        self.eta = eta
        self.iters = iterations
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'MultiClass Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained MultiClass Logistic Regression Object'
        
    def fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.unique(y) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = [] # will fill this array with binary classifiers
        
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = (y==yval) # create a binary problem whenever class is 0 (big class), y binary will be 1
            # train the binary classifier for this class
            blr = VectorBinaryLogisticRegression(self.eta,self.iters)
            blr.fit(X,y_binary)
            # add the trained classifier to the list
            self.classifiers_.append(blr)
            
        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T
        
    def predict_proba(self,X):
        probs = []
        for blr in self.classifiers_:
            probs.append(blr.predict_proba(X)) # get probability for each classifier
        
        return np.hstack(probs) # make into single matrix
    
    def predict(self,X):
        return self.unique_[np.argmax(self.predict_proba(X),axis=1)] # take argmax along row
    
#print(X)
#print(y)

In [134]:
#yhat = lr.predict(X_test)
#print('Accuracy of: ',accuracy_score(y_test,yhat))


In [135]:
class RegularizedBinaryLogisticRegression(VectorBinaryLogisticRegression):
    # extend init functions
    def __init__(self, C=0.0, **kwds):        
        # need to add to the original initializer 
        self.C = C
        # but keep other keywords
        super().__init__(**kwds) # call parent initializer
        
        
    # extend previous class to change functionality
    def _get_gradient(self,X,y):
        # call get gradient from previous class
        gradient = super()._get_gradient(X,y)
        
        # add in regularization (to all except bias term)
        gradient[1:] += -2 * self.w_[1:] * self.C
        return gradient
        

In [136]:
# now redefine the Logistic Regression Function where needed
class RegularizedLogisticRegression(LogisticRegression):
    def __init__(self, C=0.0, **kwds):        
        # need to add to the original initializer 
        self.C = C
        # but keep other keywords
        super().__init__(**kwds) # call parent initializer
        
    def fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.unique(y) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = [] # will fill this array with binary classifiers
        
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = y==yval # create a binary problem
            # train the binary classifier for this class
            blr = RegularizedBinaryLogisticRegression(eta=self.eta,
                                                      iterations=self.iters,
                                                      C=self.C)
            blr.fit(X,y_binary)
            # add the trained classifier to the list
            print(blr)
            self.classifiers_.append(blr)
            
        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T

In [137]:
lr = RegularizedLogisticRegression(eta=0.1,
                                           iterations=2500,
                                           C=.4) # get object(0.1,500)
lr.fit(X_train,y_train)
print(lr)


Binary Logistic Regression Object with coefficients:
[[-7.21034139]
 [-0.58215164]
 [-0.31597234]
 [-0.18570778]
 [-2.21767644]
 [-1.9833122 ]
 [-3.9797578 ]
 [-0.21493154]
 [-0.44603108]]
Binary Logistic Regression Object with coefficients:
[[ 4.46064138]
 [-0.21729823]
 [ 0.19244134]
 [ 0.02959124]
 [ 0.67154472]
 [ 0.63326681]
 [ 1.03332321]
 [ 0.03961603]
 [ 0.15079484]]
Binary Logistic Regression Object with coefficients:
[[-6.82342737e+00]
 [ 2.40631566e-01]
 [-5.93273242e-03]
 [ 2.37134647e-02]
 [ 1.54082619e-01]
 [ 1.18309483e-01]
 [ 2.24610276e-01]
 [ 7.38874688e-02]
 [ 2.40400657e-02]]
MultiClass Logistic Regression Object with coefficients:
[[-7.21034139e+00 -5.82151643e-01 -3.15972343e-01 -1.85707779e-01
  -2.21767644e+00 -1.98331220e+00 -3.97975780e+00 -2.14931543e-01
  -4.46031076e-01]
 [ 4.46064138e+00 -2.17298233e-01  1.92441343e-01  2.95912432e-02
   6.71544724e-01  6.33266812e-01  1.03332321e+00  3.96160322e-02
   1.50794836e-01]
 [-6.82342737e+00  2.40631566e-01 -5.9

In [138]:
yhat = lr.predict(X_test)
print('Accuracy of: ',accuracy_score(y_test,yhat))

Accuracy of:  0.6858316221765913


In [139]:
#minibatch s essentially just running regression a smaller batch. This will be much faster with hopefully similar accuracy.
from numpy import random
class StochasticGradientDescent(RegularizedLogisticRegression):
    def __init__(self,size=1000, **kwds):
        self.size = size
            #make sure to pass c, eta and iterations
        super().__init__(**kwds) # call parent initializer
        
    def fit(self, X, y):
        num_samples, num_features = X.shape
        #get a random sample 
        x_slice = X[self.size:self.size*2,:]
        y_slice = y[self.size:self.size*2]
        print(y_slice)
        self.unique_ = np.unique(y_slice) # get each unique class value
        
        num_unique_classes = len(self.unique_)
        self.classifiers_ = [] # will fill this array with binary classifiers
        #then we will
        print(np.unique(y_slice))
        #there are nans in here
        print(self.unique_)
        for i,yval in enumerate(self.unique_): # for each unique value
            print(i)
            print(yval)
            y_binary = y_slice==yval # create a binary problem
            # train the binary classifier for this class
            blr = RegularizedBinaryLogisticRegression(eta=self.eta,
                                                      iterations=self.iters,
                                                          C=self.C)
            blr.fit(x_slice,y_binary)
                # add the trained classifier to the list
            print(blr)
            self.classifiers_.append(blr)

        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T
        #stochastic takes a single sample 



In [140]:
#print(y_train)
#5194
ls = StochasticGradientDescent(eta=0.1,
                                           iterations=2500, size=1000,
                                           C=.4) # get object(0.1,500)

#print( len(y_train))

ls.fit(X_train,y_train)
#print(ls)


12936    1
47292    2
54925    2
55453    2
82736    2
        ..
69163    2
40068    1
283      1
22234    1
43395    2
Name: fit, Length: 1000, dtype: int64
[1 2 3]
[1 2 3]
0
1
Binary Logistic Regression Object with coefficients:
[[-7.79999993]
 [-0.47191441]
 [-0.32405496]
 [-0.19137874]
 [-2.0909125 ]
 [-1.92659483]
 [-3.37396253]
 [-0.21940475]
 [-0.44722669]]
1
2
Binary Logistic Regression Object with coefficients:
[[ 3.26626358]
 [-0.32558209]
 [ 0.1460727 ]
 [-0.01786393]
 [ 0.11012518]
 [ 0.18137423]
 [ 0.03496244]
 [ 0.01611119]
 [ 0.07006986]]
2
3
Binary Logistic Regression Object with coefficients:
[[-6.80188127]
 [-0.29377592]
 [-0.16439056]
 [-0.1057404 ]
 [-1.50811676]
 [-1.3758397 ]
 [-2.50865353]
 [-0.06497907]
 [-0.31569127]]


In [141]:
yhat = ls.predict(X_test)
print('Accuracy of: ',accuracy_score(y_test,yhat))


Accuracy of:  0.6858316221765913
